# Crawling Wikipedia

This notebook crawls links on Wikipedia
and visualizes the graph with NetworkX and d3.

The original can be found at

https://github.com/minrk/IPython-parallel-tutorial/tree/master/examples/wikipedia

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

enabling inline matplotlib


In [2]:
from IPython.html import widgets
from IPython.display import display
from d3networkx import ForceDirectedGraph, EventfulGraph

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [3]:
import networkx as nx

In [4]:
from IPython import parallel
rc = parallel.Client()
lbv = rc.load_balanced_view()
rc.ids

[0, 1, 2, 3]

In [5]:
%%px --local

import requests
from bs4 import BeautifulSoup

import re
wiki_pat = re.compile(r'^/wiki/([^:]*)$')


In [6]:
def links_for_page(title):
    page = BeautifulSoup(requests.get('http://en.wikipedia.org/wiki/%s' % title).text)
    links = page.find("div", id="content").findAll("a", href=wiki_pat)
    
    titles = []
    for link in links:
        title = wiki_pat.match(link['href']).group(1)
        titles.append(title)
        
    return titles

In [7]:
def add_node(g, label, **kwargs):
    """add a node to a graph, with some default fill and color"""
    kwargs.setdefault('fill', '#ccc')
    kwargs.setdefault('color', 'black')
    g.add_node(label, label=label, **kwargs)

In [8]:
d = 200

def add_links(graph, src, links):
    """Add links from src to links in graph"""
    new_nodes = []
    add_node(graph, src)
    n = len(links)
    for i,link in enumerate(links):
        if link not in graph:
            new_nodes.append(link)
            add_node(graph, link)
            
        graph.add_edge(src, link, distance=d)
    return new_nodes

In [9]:
def wikipedia_graph(lbview, root, limit=32, in_degree_limit=3):
    """build a graph by crawling Wikipedia from a root page
    
    The visualized graph will be limited to pages linked from several other pages
    """
    graph = nx.DiGraph()
    egraph = EventfulGraph()

    graph_widget = ForceDirectedGraph(egraph, width=800, height=600)
    display(graph_widget)
    
    add_node(graph, root)
    add_node(egraph, root, r=16, fill='#aef')
    surface = [root]
    while len(egraph) < limit:
        surface = [ node for node in graph if graph.out_degree(node) == 0 ]
        amr = lbview.map_async(links_for_page, surface)
        for i, links in enumerate(amr):
            src = surface[i]
            links = links[:20]
            add_links(graph, src, links)
            for node in links:
                if graph.in_degree(node) >= in_degree_limit:
                    path = nx.shortest_path(graph, root, node)
                    prv = root
                    for nxt in path[1:]:
                        if nxt not in egraph:
                            add_node(egraph, nxt)
                        egraph.add_edge(prv, nxt, distance=d)
                        egraph.node[nxt]['r'] = min(3 * graph.in_degree(nxt), 24)
                        prv = nxt
                    for parent in graph.predecessors(node):
                        if parent in egraph:
                            egraph.add_edge(parent, node, distance=d)
                    egraph.node[node]['r'] = min(3 * graph.in_degree(node), 24)
                    for child in graph.successors(node):
                        if child in egraph:
                            egraph.add_edge(node, child, distance=d)
                            egraph.node[child]['r'] = min(3 * graph.in_degree(child), 24)
                    time.sleep(0.3)
                if len(egraph) > limit:
                    return graph, egraph
            print('%s: %i' % (src, len(graph)))
            sys.stdout.flush()
    return graph, egraph
    

In [12]:
g, eg = wikipedia_graph(lbv, 'Network_science', limit=22)

Network_science: 21
Homophily: 37
Complex_contagion: 53
Reciprocity_(network_science): 53
Scale-free_network: 53
Preferential_attachment: 72
Graph_drawing: 90
Percolation_theory: 107
Balance_theory: 119
Triadic_closure: 131
Link_analysis: 149
